In [128]:
import re
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guido\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [129]:
sw = stopwords.words('portuguese')
def limpar_caracteres_especiais(string):

    limpo= re.sub(r'[^\w\s]', '', string)
    menor=limpo.lower()
    return menor


In [130]:
def separa_palavra(lista):
    palavras=[]
    for frase in lista:
        palavra=frase.split()
        for e in palavra:
            palavras.append(e)
    palavras_series=pd.Series(palavras)
    tabela_palavras=palavras_series.value_counts()
    return tabela_palavras

In [131]:
def probabilidade_frase(frase,treino):
    
    Positivo=1
    Negativo=1
    Neutro=1
    frase_split=frase.split()
    treino_positivo = treino.loc[treino.Target=='Positivo' , :]
    treino_negativo = treino.loc[treino.Target=='Negativo' , :]
    treino_neutro = treino.loc[treino.Target=='Neutro' , :]
    palavras_t=separa_palavra(treino.Mensagem)
    palavras_p=separa_palavra(treino_positivo.Mensagem)
    palavras_neg=separa_palavra(treino_negativo.Mensagem)
    palavras_neu=separa_palavra(treino_neutro.Mensagem)
    todas=len(palavras_t)
    todas_positivas=sum(palavras_p.values)
    todas_negativas=sum(palavras_neg.values)
    todas_neutras=sum(palavras_neu.values)



    for i in frase_split:
        if i not in palavras_t:
            palavras_t[i]=0
        if i not in palavras_p:
            palavras_p[i]=0

        
        if i not in palavras_neg:
            palavras_neg[i]=0    
        
        
        if i not in palavras_neu:
            palavras_neu[i]=0 
        
        
        chance_pos=(palavras_p[i]+1)/(todas+todas_positivas)
        Positivo*=chance_pos
    
        chance_neg=(palavras_neg[i]+1)/(todas+todas_negativas)
        Negativo*=chance_neg

        chance_neu=(palavras_neu[i]+1)/(todas+todas_neutras)
        Neutro*=chance_neu

    
    if Negativo>Positivo and Negativo>Neutro:
        return ('Negativo')
    elif Positivo>Negativo and Positivo>Neutro:
        return ('Positivo')
    else:
        return('Neutro')
    


In [132]:
def acur(teste,treino):
    fpos=0
    ppos=0
    fneg=0
    pneg=0
    fneu=0
    pneu=0

    mensagem_treino=treino.Mensagem
    
    mensagem_treino = [limpar_caracteres_especiais(string) for string in mensagem_treino]
    
    
    for i in range(len(mensagem_treino)):
        mensagem_treino[i] = ' '.join([k for k in mensagem_treino[i].split(" ") if k not in sw])
    
    treino.Mensagem=mensagem_treino

    
    
    target=teste.Target.tolist()
    mensagem=teste.Mensagem
    
    mensagem = [limpar_caracteres_especiais(string) for string in mensagem]
    
    
    for i in range(len(mensagem)):
        mensagem[i] = ' '.join([k for k in mensagem[i].split(" ") if k not in sw])
    
    for i in range(len(target)):
        classi=probabilidade_frase(mensagem[i],treino)
        if target[i]=='Neutro':
            if classi==target[i]:
                pneu+=1
            else:
                fneu+=1
        if target[i]=='Positivo':
            if classi==target[i]:
                ppos+=1
            else:
                fpos+=1
        if target[i]=='Negativo':
            if classi==target[i]:
                pneg+=1
            else:
                fneg+=1
    
    resp=((ppos+pneu+pneg)/(ppos+fpos+pneg+fneg+pneu+fneu))*100
    
    return(resp)


In [133]:
total = pd.read_excel('dados/Total.xlsx')

In [134]:
from sklearn.model_selection import train_test_split
#treino, teste = train_test_split(mix, test_size=prop_teste,random_state=1)

acuracia = []

for i in range(10):
    dataframe = train_test_split(total, test_size=0.3)
    # Assuming acur function calculates accuracy
    treino=dataframe[0]
    teste=dataframe[1]
    # Append the accuracy to the list
    #acuracia.append(
    print(acur(teste, treino))


78.0
80.66666666666666
76.66666666666667
75.66666666666667


KeyboardInterrupt: 